In [1]:
import pandas as pd
import numpy as np
import h2o

# Machine Learning - Using h2o Auto ML

In [2]:
full_table = pd.read_csv('./data/processed_results/osc_df')

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_231"; Java(TM) SE Runtime Environment (build 1.8.0_231-b11); Java HotSpot(TM) 64-Bit Server VM (build 25.231-b11, mixed mode)
  Starting server from /Users/nickparker/anaconda3/envs/DistributedComputing/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/kj/kg5zhyt51z7__kkfrysh9lv00000gn/T/tmp4vgqqr11
  JVM stdout: /var/folders/kj/kg5zhyt51z7__kkfrysh9lv00000gn/T/tmp4vgqqr11/h2o_nickparker_started_from_python.out
  JVM stderr: /var/folders/kj/kg5zhyt51z7__kkfrysh9lv00000gn/T/tmp4vgqqr11/h2o_nickparker_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.2
H2O cluster version age:,1 day
H2O cluster name:,H2O_from_python_nickparker_u3g2vt
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


### Train/Test Split

First Year of Existance:
- gg_drama 1943
- gg_comedy 1951
- pga 1989
- bafta 1960
- dga 1948
- sag 1995
- cannes 1970

In [4]:
# I pick a min_year where the awards shows will be relevant
min_year = 1989

# Leftover code from designing the split
# # 80/20 Split (random)
# table_sized = full_table.loc[(full_table['year'] >= min_year)]
# rand_rows = np.random.choice(table_sized.shape[0], table_sized.shape[0])
# split = int(0.8 * len(rand_rows))
# train = table_sized.iloc[rand_rows[:split]]
# valid = table_sized.iloc[rand_rows[split:]]

# # 80/20 Split (sequential)
# year_range = 2019 - min_year
# train = full_table.loc[((full_table['year'] <= 2019 - 0.2 * year_range) & (full_table['year'] >= min_year))]
# valid = full_table.loc[((full_table['year'] > 2019 - 0.2 * year_range) & (full_table['year'] < 2019))]

# # Final Prediction
# train = full_table.loc[((full_table['year'] < 2019) & (full_table['year'] >= min_year))]
# test = full_table.loc[(full_table['year'] == 2019)]

# train.to_csv('./data/processed_results/train.csv', index = False)
# valid.to_csv('./data/processed_results/valid.csv', index = False)

# H2O's Auto ML

## Pick best model by validating with the first 80% of the data

In [5]:
# 80/20 Split (sequential) Excluding 2019
year_range = 2019 - min_year
train = full_table.loc[((full_table['year'] <= 2019 - 0.2 * year_range) & (full_table['year'] >= min_year))]
valid = full_table.loc[((full_table['year'] > 2019 - 0.2 * year_range) & (full_table['year'] < 2019))]

In [6]:
train.columns

Index(['Unnamed: 0', 'year', 'film', 'wiki', 'winner', 'Oscar_win',
       'nom_gg_drama', 'winner_gg_drama', 'nom_gg_comedy', 'winner_gg_comedy',
       'nom_pga', 'winner_pga', 'nom_bafta', 'winner_bafta', 'nom_dga',
       'winner_dga', 'nom_sag', 'winner_sag', 'nom_cannes', 'winner_cannes'],
      dtype='object')

In [7]:
from h2o.automl import H2OAutoML, get_leaderboard

# Import a sample binary outcome train/test set into H2O
train = h2o.H2OFrame(train)
test = h2o.H2OFrame(valid)

# Identify predictors and response
predictors = ['year','nom_gg_drama', 'winner_gg_drama', 'nom_gg_comedy', 'winner_gg_comedy',
       'nom_pga', 'winner_pga', 'nom_bafta', 'winner_bafta', 'nom_dga', 'winner_dga',
        'nom_sag', 'winner_sag', 'nom_cannes', 'winner_cannes']

x = predictors
y = 'Oscar_win'

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=train)

# AutoML Leaderboard
lb = aml.leaderboard

# Optionally edd extra model information to the leaderboard
lb = get_leaderboard(aml, extra_columns='ALL')

# Print all rows (instead of default 10 rows)
lb.head(rows=lb.nrows)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████
22:28:14.758: Skipping training of model GBM_5_AutoML_20200121_222801 due to exception: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for GBM model: GBM_5_AutoML_20200121_222801.  Details: ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 147.0.


████████████████████████████████████████████| 100%


model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms
DeepLearning_1_AutoML_20200121_222801,0.94623,0.340217,0.72721,0.133279,0.322235,0.103836,51,0.020239
StackedEnsemble_AllModels_AutoML_20200121_222801,0.934754,0.295363,0.735027,0.133279,0.298909,0.0893465,151,0.330827
StackedEnsemble_BestOfFamily_AutoML_20200121_222801,0.915738,0.253496,0.703683,0.160984,0.277073,0.0767693,133,0.112695
XGBoost_grid__1_AutoML_20200121_222801_model_4,0.823279,0.43675,0.333895,0.217869,0.380605,0.14486,190,0.018038
XGBoost_grid__1_AutoML_20200121_222801_model_1,0.660328,0.639405,0.210021,0.233607,0.444035,0.197167,309,0.019331
XGBoost_grid__1_AutoML_20200121_222801_model_2,0.648689,0.46461,0.207111,0.204918,0.384011,0.147465,197,0.017968
XRT_1_AutoML_20200121_222801,0.587869,0.757966,0.183841,0.377049,0.440787,0.194293,61,0.029428
DRF_1_AutoML_20200121_222801,0.559344,0.762796,0.171908,0.381148,0.443478,0.196673,70,0.022369
XGBoost_3_AutoML_20200121_222801,0.397869,0.571909,0.138409,0.348361,0.416133,0.173167,176,0.016241
GBM_3_AutoML_20200121_222801,0.38918,0.819812,0.13811,0.47541,0.433233,0.187691,57,0.021485


## Once best model is picked, retrain that on the entire dataset

In [8]:
top_model = aml.leader
top_model

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_1_AutoML_20200121_222801


Status of Neuron Layers: predicting Oscar_win, 2-class classification, bernoulli distribution, CrossEntropy loss, 362 weights/biases, 9.8 KB, 1,646 training samples, mini-batch size 1


,,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
0,,1,11,Input,0,,,,,,,,,
1,,2,10,Rectifier,0,0,0,0.00171752,0.00157593,0,6.90201e-05,0.312818,0.514244,0.0674342
2,,3,10,Rectifier,0,0,0,0.00129674,0.000956646,0,-0.0167563,0.322705,0.992695,0.084619
3,,4,10,Rectifier,0,0,0,0.00520543,0.010476,0,0.0036124,0.334699,1.01982,0.088546
4,,5,2,Softmax,,0,0,0.00152497,0.00116895,0,-0.531734,1.48644,-2.21885e-05,0.0520929




ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.07744103166898385
RMSE: 0.27828228773851893
LogLoss: 0.25308096922844625
Mean Per-Class Error: 0.14196721311475402
AUC: 0.9229508196721311
AUCPR: 0.736175436066597
Gini: 0.8459016393442622

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5724147627958741: 


,,0,1,Error,Rate
0,0,120.0,2.0,0.0164,(2.0/122.0)
1,1,11.0,14.0,0.44,(11.0/25.0)
2,Total,131.0,16.0,0.0884,(13.0/147.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.55310345,0.5105405,0.8,0.0,0.0,0.9655172,1.0
1,auc,0.8783644,0.18063441,0.6708075,NaN,NaN,0.96428573,1.0
2,aucpr,0.35637513,0.4197956,0.8191254,NaN,NaN,0.25,0.0
3,err,0.16980833,0.22117023,0.2,0.5415966,0.07296227,0.03448276,0.0
4,err_count,2.3333333,3.2145503,6.0,NaN,NaN,1.0,0.0
5,f0point5,0.5180163,0.4199186,0.8396947,0.18203886,0.01279238,0.5555556,1.0
6,f1,0.8488889,0.1688304,0.88,NaN,NaN,0.6666667,1.0
7,f2,-Infinity,NaN,0.92436975,-Infinity,-Infinity,0.8333333,1.0
8,lift_top_group,6.1688223,12.773256,1.3043479,0.42666012,0.113103405,0.0,29.0
9,logloss,0.21363266,0.27688375,0.67134875,None,None,0.14638339,0.2504312



See the whole table with table.as_data_frame()

Scoring History: 


,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_r2,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2020-01-21 22:28:15,0.000 sec,None,0.000000,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,2020-01-21 22:28:15,0.448 sec,48000 obs/sec,0.979592,1,144.0,0.395181,0.475536,-0.106436,0.785902,0.383438,5.88,0.238095
2,,2020-01-21 22:28:15,0.482 sec,56758 obs/sec,11.197279,11,1646.0,0.278282,0.253081,0.451337,0.922951,0.736175,5.88,0.088435



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,winner_gg_comedy,1.000000,1.000000,0.126362
1,nom_gg_drama,0.866256,0.866256,0.109462
2,year,0.839551,0.839551,0.106087
3,nom_cannes,0.759296,0.759296,0.095946
4,winner_cannes,0.727096,0.727096,0.091877
5,winner_pga,0.721293,0.721293,0.091144
6,winner_sag,0.708602,0.708602,0.089540
7,nom_pga,0.664869,0.664869,0.084014
8,winner_gg_drama,0.645374,0.645374,0.081551
9,nom_sag,0.620017,0.620017,0.078347


In [9]:
# Re-split data on min_year to 2019
train = full_table.loc[((full_table['year'] < 2019) & (full_table['year'] >= min_year))]
test = full_table.loc[(full_table['year'] == 2019)]

# Import a binary outcome train/test set into H2O
train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [10]:
top_model.train(x=x, y=y, training_frame=train)

preds = top_model.predict(test)

preds

deeplearning Model Build progress: |██████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%


predict,p0,p1
0,0.999995,5.10505e-06
0,0.999828,0.000171615
0,0.999995,5.23408e-06
0,0.999991,8.71287e-06
0,0.999995,5.10505e-06
0,0.999991,8.71287e-06
0,0.9429,0.0571003
0,0.999851,0.000149178
0,0.999989,1.05672e-05


## Predict the winner

In [11]:
test['pred'] = preds['predict']
test['probA'] = preds['p1']
test_pd = test.as_data_frame(use_pandas=True)

In [12]:
test_pd[['film','probA']].sort_values('probA', ascending = False)

,film,probA
6,1917 (2019 film),0.057100
1,The Irishman,0.000172
7,Once Upon a Time in Hollywood,0.000149
8,Parasite (2019 film),0.000011
3,Joker (2019 film),0.000009
5,Marriage Story,0.000009
2,Jojo Rabbit,0.000005
0,Ford v Ferrari,0.000005
4,Little Women (2019 film),0.000005


In [13]:
test_pd.T #The DGA's (1/26) and BAFTA's (2/2) Haven't come out yet

,0,1,2,3,4,5,6,7,8
Unnamed: 0,554,555,556,557,558,559,560,561,562
year,2019,2019,2019,2019,2019,2019,2019,2019,2019
film,Ford v Ferrari,The Irishman,Jojo Rabbit,Joker (2019 film),Little Women (2019 film),Marriage Story,1917 (2019 film),Once Upon a Time in Hollywood,Parasite (2019 film)
wiki,/wiki/Ford_v_Ferrari,/wiki/The_Irishman,/wiki/Jojo_Rabbit,/wiki/Joker_(2019_film),/wiki/Little_Women_(2019_film),/wiki/Marriage_Story,/wiki/1917_(2019_film),/wiki/Once_Upon_a_Time_in_Hollywood,/wiki/Parasite_(2019_film)
winner,False,False,False,False,False,False,False,False,False
Oscar_win,0,0,0,0,0,0,0,0,0
nom_gg_drama,0,1,0,1,0,1,1,0,0
winner_gg_drama,0,0,0,0,0,0,1,0,0
nom_gg_comedy,0,0,1,0,0,0,0,1,0
winner_gg_comedy,0,0,0,0,0,0,0,1,0
